In [1]:
import sys

import numpy as np
from process_bigraph import pp


sys.path.insert(0, '..')


from biosimulator_processes.services.rest_service import BiosimulationsRestService

/Users/alexanderpatrie/Library/Caches/pypoetry/virtualenvs/biosimulator-processes-JaN5cQMh-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CobraProcess registered successfully.
CopasiProcess registered successfully.
_CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
AmiciProcess registered successfully.
ODEComparatorProcess not available. Error: cannot import name 'fetch_sbml_file' from 'biosimulator_processes.io' (/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/io.py)
GetSbmlStep not available. Error: cannot import name 'fetch_sbml_file' from 'biosimulator_processes.io' (/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/io.py)
CompositionPlotter registered successfully.
Plotter2d registered successfully.
ODEComparatorStep registered successfully.


In [2]:
repressilator_run_id = "61fea4a08c1e3dc95a79802e"

query = "tumor"

result = await BiosimulationsRestService().extract_data(query)

Failed to fetch simulation run files:
Expecting value: line 1 column 1 (char 0)


AttributeError: 'NoneType' object has no attribute 'run_id'

In [ ]:
import requests
get_files_url = f'https://api.biosimulations.dev/results/{repressilator_run_id}/download'
headers = {'accept': 'application/json'}
files_resp = requests.get(get_files_url, headers=headers)

In [ ]:
files_resp.json()

In [2]:
import h5py

def explore_hdf5(file_path):
    with h5py.File(file_path, 'r') as f:
        def print_attrs(name, obj):
            print(f"Name: {name}")
            print("Attributes:")
            for key, val in obj.attrs.items():
                print(f"    {key}: {val}")
            if isinstance(obj, h5py.Dataset):
                print(f"Dataset shape: {obj.shape}")
                print(f"Dataset dtype: {obj.dtype}")
            elif isinstance(obj, h5py.Group):
                print("Group")
            print()
        
        f.visititems(print_attrs)

# Usage example
file_path = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/reports.h5'
explore_hdf5(file_path)

Name: simulation.sedml
Attributes:
    combineArchiveLocation: simulation.sedml
    uri: simulation.sedml
Group

Name: simulation.sedml/Figure_1c
Attributes:
    _type: SedPlot2D
    sedmlDataSetDataTypes: ['float64' 'float64' 'float64' 'float64']
    sedmlDataSetIds: ['data_gen_px' 'data_gen_py' 'data_gen_pz' 'data_gen_time']
    sedmlDataSetLabels: ['data_gen_px' 'data_gen_py' 'data_gen_pz' 'data_gen_time']
    sedmlDataSetNames: ['LacI protein' 'TetR protein' 'cI protein' 'time']
    sedmlDataSetShapes: ['601' '601' '601' '601']
    sedmlId: Figure_1c
    sedmlName: Figure 1c
    uri: simulation.sedml/Figure_1c
Dataset shape: (4, 601)
Dataset dtype: float64

Name: simulation.sedml/report
Attributes:
    _type: SedReport
    sedmlDataSetDataTypes: ['float64' 'float64' 'float64' 'float64' 'float64' 'float64' 'float64']
    sedmlDataSetIds: ['data_set_time' 'data_set_laci_protein' 'data_set_tetr_protein'
 'data_set_ci_protein' 'data_set_laci_mrna' 'data_set_tetr_mrna'
 'data_set_ci_mrn

In [3]:
def read_dataset(file_path, dataset_id):
    with h5py.File(file_path, 'r') as f:
        if dataset_id in f:
            data = f[dataset_id][:]
            print(f"Data from dataset '{dataset_id}':\n{data}")
        else:
            print(f"Dataset '{dataset_id}' not found in the file.")
            
            
read_dataset(file_path, 'data_set_time')

Dataset 'data_set_time' not found in the file.


In [4]:
from biosimulator_processes.services.rest_service import BiosimulationsRestService

In [5]:
from basico import * 


fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/BIOMD0000000012_url.xml'

model = load_model(fp)

In [ ]:
get_species(model=model)

In [ ]:
from biosimulator_processes.processes.copasi_process import CopasiProcess
group_path = 'simulation.sedml/report'
dataset_label = 'cI protein'
outputs = BiosimulationsRestService().read_report_outputs(file_path, dataset_label)


for datum in outputs.data:
    print(datum.dataset_label)

process = CopasiProcess(config={'model': {'model_source': fp}})

In [ ]:
process.initial_state()['floating_species_concentrations'].keys()

In [ ]:
def is_equal(a, b):
    return a == b or b == a  


from dataclasses import dataclass

from process_bigraph import Process


@dataclass 
class OutputAspectVerification:
    aspect_type: str  # one of: 'names', 'values'. TODO: Add more
    is_verified: bool
    

def create_ode_process_instance(process_name: str, biomodel_id=None, sbml_model_file=None) -> Process:
    module_name = f'{process_name}_process'
    import_statement = f'biosimulator_processes.processes.{module_name}'
    module_paths = module_name.split('_')
    class_name = module_paths[0].replace(module_name[0], module_name[0].upper())
    class_name += module_paths[1].replace(module_paths[1][0], module_paths[1][0].upper())
    module = __import__(
        import_statement, fromlist=[class_name])
    model_source = biomodel_id or sbml_model_file
    bigraph_class = getattr(module, class_name)
    return bigraph_class(config={'model': {'model_source':model_source}})


def verify_ode_process_output_names(process_name: str, source_report_fp: str, biomodel_id: str = None, sbml_model_file: str = None) -> OutputAspectVerification:
    # Get the class from the module
    # TODO: Automatically generate this from the biosimulations rest api
    process = create_ode_process_instance(process_name, biomodel_id, sbml_model_file)
    process_keys = list(process.inputs()['floating_species_concentrations'].keys())
    
    report_outputs = BiosimulationsRestService().read_report_outputs(report_file_path=source_report_fp)
    report_keys = [datum.dataset_label for datum in report_outputs.data]
    for i, val in enumerate(report_keys):
        if report_keys[i].lower() == 'time':
            report_keys.pop(i)
    
    return OutputAspectVerification(aspect_type='names', is_verified=is_equal(report_keys, process_keys))

In [6]:
from biosimulator_processes.verify.exec import verify_ode_process_output_names


verify_ode_process_output_names(process_name='copasi', source_report_fp=file_path, sbml_model_file=fp)

found a filepath


OutputAspectVerification(aspect_type='names', is_verified=True, expected_data=['LacI protein', 'TetR protein', 'cI protein', 'LacI mRNA', 'TetR mRNA', 'cI mRNA'], process_data=['LacI protein', 'TetR protein', 'cI protein', 'LacI mRNA', 'TetR mRNA', 'cI mRNA'])

In [ ]:
x = np.array([1, 2, 3])
y = np.array([1, 2, 3.0000001])

In [ ]:
np.allclose(x, y)